# Session 4 – Peak Fitting for Spectroscopy

This notebook demonstrates how to fit specific lineshapes to peaks in spectroscopic data. You will be introduced to Gaussian, Lorentzian and Voigt peak shapes and shown different methods for assessing goodness of fit.

Authors:
- Tom Slater

Email: slatert2@cardiff.ac.uk

## Learning objectives

By the end you will be able to:
- Generate and visualise Gaussian, Lorentzian and Voigt lineshapes (common to spectroscopic data).
- Quantify model–data mismatch with **root-mean-square (RMS) error**.
- Perform basic parameter estimation by **minimising RMS error**.
- Fit using scipy's **curve_fit()** and assess goodness of fit through different measures.



## 0. Setup

Run the cell below to import the libraries used throughout. We will only use standard scientific Python packages.


In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt


## 1. Lineshape functions

In spectroscopy and diffraction, two common peak shapes are the **Gaussian** and the **Lorentzian**. The definitions of these are:

- **Gaussian** (standard deviation `sigma`):  
  $G(x; A, x_0, \sigma) = A\, \exp\!\left(-\frac{(x-x_0)^2}{2\sigma^2}\right)$

- **Lorentzian** (half-width at half-maximum `gamma`):  
  $L(x; A, x_0, \gamma) = \frac{A\, \gamma^2}{(x-x_0)^2 + \gamma^2}$

Conventions vary: some definitions normalise the area to 1. Here we keep **amplitude `A`** as the peak height for both models (i.e. value at the centre `x_0`).

**Design choice:** We'll use the consistent parameter order `(x, A, x0, width)` where width is `sigma` for Gaussian and `gamma` for Lorentzian.


The following code demonstrates a function that defines a Gaussian:

In [ ]:
def gaussian(x, A, x0, sigma):
    """Return a Gaussian lineshape evaluated at x.

    Parameters
    ----------
    x : array_like
        Coordinates at which to evaluate the function.
    A : float
        Peak amplitude (height at the centre x0).
    x0 : float
        Peak centre.
    sigma : float
        Standard deviation (controls width).

    Returns
    -------
    ndarray
        G(x; A, x0, sigma).
    """
    x = np.asarray(x)
    return(A * np.exp(-((x - x0) ** 2) / (2.0 * sigma ** 2)))

<div class="alert alert-success">
<b>Task 1.1: Write a function called `lorentzian` that returns a numpy array containing a Lorentzian distribution. The function should have a readable docstring (similar to the gaussian() function above).<br></b>

</div>

In [ ]:
#Input your answer to 1.1 here.



### Quick check

Let's visualise both shapes for the same nominal parameters. The plots below use `matplotlib` to plot the numpy arrays generated by the two functions, in a manner you have seen before.

In [ ]:
x = np.linspace(-5, 5, 501) #This line creates a numpy array from -5 to +5 with 500 elements.

G = gaussian(x, A=1.0, x0=0.0, sigma=1.0) #Use the gaussian function to output an array, G.
L = lorentzian(x, A=1.0, x0=0.0, gamma=1.0) #Use the lorentzian function to output an array, L.

plt.figure()
plt.plot(x, G, label="Gaussian (A=1, x0=0, sigma=1)")
plt.plot(x, L, label="Lorentzian (A=1, x0=0, gamma=1)")
plt.xlabel("x"); plt.ylabel("Intensity"); plt.legend(); plt.title("Gaussian vs Lorentzian")
plt.show()

## 2. Fitting lineshapes to data
There are many cases in chemistry in which we may obtain data and would like to fit a function to that data. In the case of the Gaussian and Lorentzian functions above, fitting the functions allows us to determine the peak position, amplitude and width, which may give us important information on the structure of molecules and surfaces.

Fitting a function to data generally involves minimising the distance between the function and the data. One common method of defining the distance between two datasets is the **root-mean-square (RMS) error**.


### 2.1. RMS error between two datasets

Given two arrays of equal length, the **root-mean-square (RMS) error** is:

$\mathrm{RMS}(y, \hat{y}) = \sqrt{\frac{1}{N} \sum_{i=1}^N (y_i - \hat{y}_i)^2 }$

The code below defines a function for calculating the RMS between two datasets (y and yhat).


In [ ]:
def rms_error(y, yhat):
    """Compute the root-mean-square error between y and yhat.

    Parameters
    ----------
    y : array_like
        Ground-truth or observed data.
    yhat : array_like
        Model predictions at the same x-values.

    Returns
    -------
    float
        RMS error.
    """
    y = np.asarray(y)
    yhat = np.asarray(yhat)
    if y.shape != yhat.shape:
        raise ValueError("y and yhat must have the same shape")
    rmse_value = float(np.sqrt(np.mean((y - yhat) ** 2)))
    return(rmse_value)


<div class="alert alert-success">
<b>Task 2.1: Check that the code above returns a reasonable value for two identical datasets. Create two arrays where `y == yhat` and confirm the RMS error is 0. <br></b>
Hint: You could create two lists and convert this to a numpy array using np.asarray().
</div>

In [ ]:
#Input your answer to 2.1 here.


<div class="alert alert-success">
<b>Task 2.2: Create two arrays of the same length, but with different values, and calculate the RMS error between the two arrays. <br></b>
How has the RMS error changed from the previous task? 
</div>

In [ ]:
#Input your answer to 2.2 here.


### 2.2 Simulating experimental data
We'll now create a noisy single-peak dataset that imitates an experimental spectrum, so that we can practice fitting to some data.

The code below creates a Gaussian dataset and then adds noise using the `np.random.default_rng()` function.

In [ ]:
#Generate noise
rng = np.random.default_rng(42)

# x-axis for our synthetic spectrum
x = np.linspace(-5, 5, 751)

# Ground-truth parameters
A_true   = 1.2
x0_true  = -0.6
w_true   = 0.8    # sigma if Gaussian, gamma if Lorentzian
noise_sd = 0.05

#Generate Gaussian peak
y_clean = gaussian(x, A_true, x0_true, w_true)

#Add noise to the data
y_obs = y_clean + rng.normal(0.0, noise_sd, size=x.shape)

plt.figure()
plt.plot(x, y_obs, '.', ms=3, label='Observed (noisy)')
plt.plot(x, y_clean, label='True (clean)')
plt.xlabel("x"); plt.ylabel("Intensity"); plt.title("Synthetic dataset"); plt.legend()
plt.show()



## 2.3 Fit by minimising RMS error (grid search)

Now you have generated some data, we can try to fit a function.

We will write a generic **grid-search** minimiser that explores a parameter space and returns the parameters that give the smallest RMS error. This is robust, dependency-free, and easy to understand, though not the fastest.

We'll fit both Gaussian and Lorentzian models to the same data and compare RMS errors.

Let's go through the process step-by-step to set this up.


Firstly, we'll need to define the limits of each parameter to search through and then calculate the RMS error for each value in our search range. We'll use an `if` statement to check if the RMS value is lower than the best value we have so far, and if it is we'll update the best RMS value and best parameters.

The example code below demonstrates this for a single parameter (A).

In [ ]:
A_array = np.linspace(0.1, 2.0, 51)
x0 = -0.6
width = 0.8

best_rms = np.inf
best = None

for A in A_array:
    params = (A, x0, width)
    yhat = gaussian(x, A, x0, width)
    r = rms_error(y_obs, yhat)
    if r < best_rms:
        best_rms = r
        best = params

print("The best RMS value found is: " + str(best_rms) + " with parameters: " + str(best))

<div class="alert alert-success">
<b>Task 2.3: Use a nested for loop to scan over parameters for A, x0 and width to fit a Gaussian to the y_obs data. Remember, a nested for loop is a for loop within another for loop.<br><br>
    
Choose suitable ranges for each parameter, with 51 values in each range for each. Output the best RMS value as best_rms_gaussian and the best parameters as best_gaussian.<br></b>
How long does the search take? What happens to the time and the best fit parameters if you use only 21 points in the parameter search instead of 51?
</div>

In [ ]:
#Input your answer to 2.3 here.


<div class="alert alert-success">
<b>Task 2.4: Repeat Task 2.3, but fit the best parameters for a Lorentzian function instead. Output the best RMS value as best_rms_lorentzian and the best parameters as best_lorentzian.<br></b>
</div>

In [ ]:
#Input your answer to 2.4 here.


The Gaussian fit should give the lowest RMS value, as the original data was Gaussian in nature.


### Visualise the fits


We know that the model data is Gaussian in shape, so the best fit should be from the Gaussian function. Let's visually check that by plotting the two fitted functions with the y_obs data.

In [ ]:
plt.figure()
plt.plot(x, y_obs, '.', ms=3, label='Observed')
plt.plot(x, gaussian(x, best_gaussian[0],best_gaussian[1],best_gaussian[2]), label=f'Gaussian fit (RMS={best_rms_gaussian:.4f})')
plt.plot(x, lorentzian(x, best_lorentzian[0],best_lorentzian[1],best_lorentzian[2]), label=f'Lorentzian fit (RMS={best_rms_lorentzian:.4f})')
plt.xlabel("x"); plt.ylabel("Intensity"); plt.title("Model fits by RMS minimisation"); plt.legend()
plt.show()

### Residuals

One other method for assessing goodness of fit is to visually inspect the residuals of the fit. The residuals are the difference between the observed and fitted data, and you can obtain them by simply subtracting the fitted data from the original data. The code below calculates the residuals for each of the two fits, and then plots the residuals below each fit.

In [ ]:
#Calculating the residuals for each fit
residuals_gaussian = y_obs - gaussian(x, best_gaussian[0],best_gaussian[1],best_gaussian[2])
residuals_lorentzian = y_obs - lorentzian(x, best_lorentzian[0],best_lorentzian[1],best_lorentzian[2])

fig, axs = plt.subplots(2, 2)

# Top plot: data and fit
axs[0,0].plot(x, y_obs, '.', ms=3, label='Observed')
axs[0,0].plot(x, gaussian(x, best_gaussian[0],best_gaussian[1],best_gaussian[2]), label=f'Gaussian fit (RMS={best_rms_gaussian:.4f})')
axs[0,0].set_ylabel("Intensity")
axs[0,0].set_title("Gaussian fit (minimize RMS)")

# Bottom plot: residuals
axs[1,0].axhline(0, color='k', lw=1)
axs[1,0].plot(x, residuals_gaussian, '.', ms=3, color="red")
axs[1,0].set_xlabel("x")
axs[1,0].set_ylabel("Residuals")

# Top plot: data and fit
axs[0,1].plot(x, y_obs, '.', ms=3, label='Observed')
axs[0,1].plot(x, lorentzian(x, best_lorentzian[0],best_lorentzian[1],best_lorentzian[2]), label=f'Gaussian fit (RMS={best_rms_gaussian:.4f})')
axs[0,1].set_ylabel("Intensity")
axs[0,1].set_title("Lorentzian fit (minimize RMS)")

# Bottom plot: residuals
axs[1,1].axhline(0, color='k', lw=1)
axs[1,1].plot(x, residuals_lorentzian, '.', ms=3, color="red")
axs[1,1].set_xlabel("x")
axs[1,1].set_ylabel("Residuals")

plt.tight_layout()
plt.show()

You can see that there are larger residuals for the Lorentzian fit above, demonstrating where the lineshape does not match the data.

## 3. Voigt profiles
The Gaussian and Lorentzian lineshapes used so far fit to many different types of data. However, there are many types of spectroscopy in which lineshapes become a little more complicated.


In X-ray photoelectron spectroscopy (XPS), and many other types of spectroscopy, peaks take a shape known as a Voigt profile, which is a convolution of a Gaussian and Lorentzian function. This shape often arises due to combinations of Gaussian and Lorentzian peak broadening processes.

Thankfully, rather than having to code a convolution of two functions, scipy provides us with a function to generate a Voigt profile. The `voigt_profile()` function takes 3 arguments: the array of x values, the sigma of the Gaussian distribution and the gamma of the Lorentzian distribution.

In [ ]:
from scipy.special import voigt_profile

In [ ]:
y_voigt = voigt_profile(x, 1.0, 1.0)

plt.figure()
plt.plot(x, y_voigt)
plt.show()

The example above had both sigma and gamma equal to 1. By changing the relative values, you can change the weighting of the Gaussian and Lorentzian components. Setting either sigma or gamma to 0 results in a profile that is equal to a pure Lorentzian or Gaussian distribution, respectively.

In [ ]:
y_gauss = voigt_profile(x, 1.0, 0)
y_lorentz = voigt_profile(x, 0, 1.0)
plt.figure()
plt.plot(x, y_gauss, label='Gaussian distribution')
plt.plot(x, y_lorentz, label='Lorentzian distribution')
plt.legend()
plt.show()

The `voigt_profile()` function does not have any way to change the amplitude or position of the peak. The distribution is normalised so that the intensity under the peak is equal to 1 and it is centred at zero. 

To specify an amplitude and position, it's necessary to multiply the Voigt profile by a value to define an amplitude, and subtract a value from x to specify a position. As the area under the peak in `voigt_profile` is 1, the area in the code below is just the number which you multiply by. For example, to plot a Voigt profile with amplitude approximately 1, an area of 5 and a centre at 0.2:

In [ ]:
y_voigt = 5*voigt_profile(x-0.2, 1.0, 1.0)

plt.figure()
plt.plot(x, y_voigt)
plt.show()

<div class="alert alert-success">
<b>Task 3.1: Write a function called `shifted_voigt` that takes the arguments (x, area, centre, sigma, gamma) and returns a Voigt profile with the area and centre specified in the arguments. <br></b>
Hint: Modify the first line of the code cell above to use in your function.
</div>

In [ ]:
#Input your answer to 3.1 here.


## 4. Fitting multi-peak models
If we consider XPS data further, it's very rare that datasets will contain a single peak. Typically, there will be two or more (sometimes overlapping) peaks in a spectrum.

Creating a model with two peaks is fairly simple. You can do this by just running a for loop and adding created Voigt profiles to a dataset.

For example, for 2 peaks with different centres and amplitudes:

In [ ]:
n_peaks = 2
params = [1.0, 2.5, 1.0, 1.0, 2.0, -2.5, 1.0, 1.0] #Need 4 parameters per peak, so 8 in total here.

y = np.zeros_like(x)

for i in range(n_peaks):
    #In the line below, the index of params is 4*i as there are 4 parameters per peak.
    y += shifted_voigt(x, params[4*i],params[4*i+1], params[4*i+2], params[4*i+3])
    print(params[4*i])

plt.figure()
plt.plot(x, y)
plt.show()

<div class="alert alert-success">
<b>Task 4.1: Write a function called `multi_voigt` that takes the arguments (x, *params) and returns a profile with multiple Voigt peaks. <br></b>
Note: *params here specifies that multiple arguments could be passed as params, which will be combined into a tuple.<div>
Hint: Think about how you would define n_peaks in your function based on the input params. Again, use the code in the cell above as the basis for your function.
</div>



In [ ]:
#Input your answer to 4.1 here.


Now, let's try to fit two Voigt peaks to the data we've generated above, to recover our input values.

Previously, we fit lineshapes by minimising the RMS error in a nested for loop. If we tried that now, we'd have 8 nested for loops (one for each parameter) and the process would take a considerable amount of time!

Instead, we'll use a built-in scipy function called `curve_fit()`. The `curve_fit()` function takes the function to fit, the x values, the data and the parameters of the function as arguments, and returns two items:

- popt (the parameters associated with the optimal fit)
- pcov (the approximate covariance of the parameters)

Let's see an example:

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
# Initial guess for [A1, x01, sigma1, gamma1, A2, x02, sigma2, gamma2]
p0 = [1.0, -1.5, 1.0, 0.5, 1.0, 2.5, 1.0, 0.5]

#Calculate the optimal parameters and the covariance of parameters.
popt, pcov = curve_fit(multi_voigt, x, y, p0=p0)

print("Best-fit parameters:", popt)

#Generate a profile based on the optimal parameters in popt
y_fit = multi_voigt(x, *popt)

plt.plot(x, y, '.', ms=3, label="Data") #Plot the original data
plt.plot(x, y_fit, '-', lw=2, label="Fit") #Plot the fitted function
plt.legend(); plt.title("Fitted two Voigt peaks with curve_fit")
plt.show()

The fit above appears perfect, and we might expect that considering the fitted values match our initial values.

We can assess the goodness of fit in a number of ways.

Firstly, we can examine the covariance of parameters from `curve_fit()`.

In [ ]:
print(pcov)

At first glance, the array of numbers in pcov are not that easy to interpret. As the pcov array contains covariances, the diagaonals should be the variance on each of the parameters (don't worry about this, but it's true!). The standard deviation on each parameter can be calculated by taking the square root of the diagonal elements.

In [ ]:
perr = np.sqrt(np.diag(pcov))
print(perr)

The standard deviations on each parameter are very small relative to the parameter values, as we'd expect!

We could also calculate the RMS error between the initial data and the fitted function to estimate goodness of fit, by using the RMS error function we defined previously.

In [ ]:
print(rms_error(y, y_fit))

The RMS error here is vanishingly small, as we'd expect here.

<div class="alert alert-success">
<b>Task 4.2: Attempt to fit 3 peaks to the y data obtained above for two Voigt peaks, with appropriate initial guesses for all parameters. Plot the residual and calculate both the RMS error and the standard deviations on all parameters.<br></b>
How do the errors compare to the correct fitting of two peaks?
</div>


In [ ]:
#Input your answer to 4.2 here.
